# Fetch page content

In [1]:
from agents import Agent, function_tool, Runner
import requests

def fetch_url(url):
    jina_reader_base_url = 'https://r.jina.ai/'
    jina_reader_url = jina_reader_base_url + url
    response = requests.get(jina_reader_url)
    return response.content.decode('utf8')
    
content = fetch_url('https://en.wikipedia.org/wiki/Capybara')

In [2]:
import requests
from typing import Optional

reader_url_prefix = "https://r.jina.ai/"

def get_page_content(url: str) -> Optional[str]:
    """
    Fetch the Markdown content of a web page using the Jina Reader service.

    This function prepends the Jina Reader proxy URL to the provided `url`,
    sends a GET request with a timeout, and decodes the response as UTF-8 text.

    Args:
        url (str): The URL of the page to fetch.

    Returns:
        Optional[str]: The Markdown-formatted content of the page if the request
        succeeds; otherwise, None.

    Raises:
        None: All network or decoding errors are caught and suppressed.
               Logs or error messages could be added as needed.
    """
    reader_url = reader_url_prefix + url

    try:
        response = requests.get(reader_url, timeout=10)
        response.raise_for_status()  # raises for 4xx/5xx HTTP errors
        return response.content.decode("utf-8")
    except (requests.exceptions.RequestException, UnicodeDecodeError) as e:
        # Optional: log or print the error for debugging
        print(f"Error fetching content from {url}: {e}")
        return None

# Summary Agent

In [3]:
# tool for agent to read wikipea content

from agents import function_tool

@function_tool
def fetch_wikipedia_content(url: str) -> str:
    """
    Fetches and returns the Markdown content of a Wikipedia page.
    """
    wiki_path = url.replace("https://", "")
    content = get_page_content(wiki_path)
    if content is None:
        return "Could not fetch content from Wikipedia."
    return content


In [4]:
# Tool for agent to save summary

@function_tool
def save_summary(title: str, summary: str) -> str:
    """
    Save a summary to disk for later retrieval.
    """
    from pathlib import Path

    summaries_dir = Path("./summaries")
    summaries_dir.mkdir(exist_ok=True)
    file_path = summaries_dir / f"{title}.txt"
    file_path.write_text(summary, encoding="utf-8")
    return f"Saved summary to {file_path}"

In [5]:
summarizing_instructions = """
You are a helpful assistant. When a user asks about a Wikipedia page, follow these steps:

1. Identify the URL in the user's request.
2. Use the 'Fetch wikipedia content' tool to get the Markdown content of the page.
3. Read the content and generate a concise, informative summary of the page.
4. Use the 'Save Summary' tool to save the summary to disk.
5. Respond to the user confirming that the summary has been saved, 
   and optionally include a brief preview of the summary.

Always make sure to handle failures gracefully: if fetching the page or saving the summary fails, inform the user.
"""

content_tools = [
    fetch_wikipedia_content,
    save_summary
]

summarizing_agent = Agent(
    name='summarizing_agent',
    handoff_description='use this agent when you need a summary of a Wikipedia page',
    tools=content_tools,
    instructions=summarizing_instructions,
    model='gpt-4o-mini'
)


In [6]:
from toyaikit.chat.runners import OpenAIAgentsSDKRunner
from toyaikit.chat import IPythonChatInterface

chat_interface = IPythonChatInterface()

runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=summarizing_agent
)

In [7]:
await runner.run();

You: stop


Chat ended.


# Search Function

In [8]:
# search/index class


class SearchTools:

    def __init__(self) -> None:
        self.index = AppendableIndex(text_fields=['content'])

    @function_tool
    def index_content(self, url: str, content: Optional[str] = None) -> str:
        """
        Index the content of a Wikipedia page into the search index.
        """
        if content is None:
            content = fetch_wikipedia_content(url)

        if content.startswith("Error"):
            return content

        # Split into chunks for indexing
        chunks = [{"content": content[i:i+3000]} for i in range(0, len(content), 3000)]
        for chunk in chunks:
            chunk["url"] = url
            self.index.append(chunk)

        return f"SUCCESS: Indexed {url}"


In [9]:
from typing import Any, Dict, List, Optional

@function_tool
def search(self, query: str) -> List[Dict[str, Any]]:
    """
    Search the index for documents matching a query string.
    """
    return self.index.search(query, num_results=5)

In [10]:
from minsearch import AppendableIndex
from toyaikit.tools import wrap_instance_methods

search_tools = SearchTools()
search_tool_methods = wrap_instance_methods(function_tool, search_tools)

search_instructions = """
Your task is to search through indexed Wikipedia pages.

Before performing a search:
1. Check if the page has been indexed.
2. If not, call `index_content` to index it first.

Important rules:
- Do NOT call `search` using a Wikipedia URL as the query.
- After successfully indexing a page, respond:
  "The page has been indexed. What would you like to learn about it?"
- Only perform a search if the user provides a natural-language question or topic.
- Do not attempt to summarize content in the search agent.
"""


In [11]:
search_agent = Agent(
    name='search_agent',
    handoff_description='use this agent to search indexed Wikipedia pages',
    tools=search_tool_methods,
    instructions=search_instructions,
    model='gpt-4o-mini'
)


In [12]:
search_runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=search_agent
)

await search_runner.run();

You: stop


Chat ended.


# Triaging Agent

In [13]:
from agents import handoff, Agent

triage_instructions = """
You are the orchestrator between two specialized agents:

1. summarizing_agent — summarizes Wikipedia pages.
2. search_agent — searches within previously indexed pages to answer content questions.

Routing rules:
- If the user sends a Wikipedia link without any question → hand off to summarizing_agent.
- If the user asks for a summary or overview of a page → summarizing_agent.
- If the user asks a follow-up question about content that has already been indexed, 
  including any natural-language question like "What are threats to capybara populations?" → hand off to search_agent.
- Always use search_agent for detailed or specific content queries that relate to already indexed pages.
"""

triage_agent = Agent(
    name="triage_agent",
    instructions=triage_instructions,
    handoffs=[
        handoff(summarizing_agent, on_handoff=lambda ctx: print('summarizing handoff called')),
        handoff(search_agent, on_handoff=lambda ctx: print('search handoff called')),
    ],
    model='gpt-4o-mini'
)


In [14]:
chat_interface = IPythonChatInterface()

triage_runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=triage_agent
)

In [ ]:
await triage_runner.run();

You: index this page https://en.wikipedia.org/wiki/Caviodon


summarizing handoff called
handoff: transfer_to_summarizing_agent
handoff: summarizing_agent -> triage_agent successful


You: What are threats to capybara populations?


search handoff called
handoff: transfer_to_search_agent
handoff: search_agent -> triage_agent successful


You: What are threats to capybara populations?


search handoff called
handoff: transfer_to_search_agent
handoff: search_agent -> triage_agent successful
